In [24]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import  SVC, NuSVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
import yandex_evaluation as evaluation
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [25]:
tr = pd.read_csv('input/training.csv', index_col='id')

In [26]:
variables = tr.columns[:-4]

In [27]:
# m = SVC(
#     C=1.0,
#     kernel='rbf',
#     degree=3,
#     gamma=0.0,
#     coef0=0.0,
#     shrinking=True,
#     probability=True,
#     tol=0.001,
#     cache_size=1000,
#     class_weight=None,
#     verbose=False,
#     max_iter=-1,
#     random_state=None)

In [28]:
# m = KNeighborsClassifier(
#     n_neighbors=10,
#     weights='distance',
#     algorithm='auto',
#     leaf_size=30,
#     p=2,
#     metric='minkowski',
#     metric_params=None)

In [29]:
RadiusNeighborsClassifier(
    radius=1.0,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    outlier_label=None,
    metric_params=None)

RadiusNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, outlier_label=None, p=2, radius=1.0,
             weights='uniform')

In [30]:
m.fit(tr[variables], tr.signal)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=10, p=2, weights='distance')

In [31]:
check_agreement = pd.read_csv('input/check_agreement.csv', index_col='id')
agreement_probs = m.predict_proba(check_agreement[variables])[:, 1]
ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print 'KS metric', ks, ks < 0.09

KS metric 0.0312621389384 True


In [32]:
check_correlation = pd.read_csv('input/check_correlation.csv', index_col='id')
correlation_probs = m.predict_proba(check_correlation[variables])[:, 1]
cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print 'CvM metric', cvm, cvm < 0.002

CvM metric 0.00102007033103 True


In [33]:
tr_eval = tr[tr['min_ANNmuon'] > 0.4]
tr_probs = m.predict_proba(tr_eval[variables])[:, 1]
AUC = evaluation.roc_auc_truncated(tr_eval['signal'], tr_probs)
print 'AUC', AUC

AUC 1.0


In [34]:
test = pd.read_csv('input/test.csv', index_col='id')
result = pd.DataFrame({'id': test.index})
result['prediction'] = m.predict_proba(test[variables])[:, 1]

In [35]:
result.to_csv('output/submission_001.csv', index=False, sep=',')